In [1]:
import pandas as pd
import torch
import json
import re

In [2]:
import sys

sys.path.append('../utils')
from evaluator import Evaluator

from json_format import TextJSONProcessor

json_proc = TextJSONProcessor()

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/FRED-T5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/FRED-T5-large")
# tokenizer = AutoTokenizer.from_pretrained('fredT5-large-checkpoints/checkpoint-000')
# model = AutoModelForSeq2SeqLM.from_pretrained('fredT5-large-checkpoints/checkpoint-2000').to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
converters = {'json': json.loads}
train = pd.read_csv('../data/train_9k_valid.csv', index_col=0, converters=converters).sample(frac=1, random_state=42)
val_set = pd.read_csv('../data/val_set_300_sb_valid.csv', index_col=0, converters=converters).sample(frac=1, random_state=42)
manual_test = pd.read_csv('../data/manual_test_100.csv', index_col=0, converters=converters).sample(frac=1, random_state=42)

In [5]:
tokenizer.add_tokens(json_proc.spec_tokens)
model.resize_token_embeddings(len(tokenizer))


assert train.loc[train.index[0], 'json'] == json_proc.unprocess_json(json_proc.process_json(train.loc[train.index[0], 'json']))

In [6]:
from datasets import Dataset

ads_dataset = Dataset.from_pandas(train[["Text", "json"]])
ads_dataset = ads_dataset.train_test_split(test_size=0.005, seed=42)
ads_dataset = ads_dataset.flatten()
ads_dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 8766
    })
    test: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 45
    })
})

In [7]:
def preprocess_function(examples):
    inputs = [tokenizer.bos_token + text + tokenizer.eos_token for text in examples["Text"]]
    targets = [tokenizer.bos_token + json_proc.process_json(bundles) + tokenizer.eos_token for bundles in examples["json"]]
    # model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True)
    # return model_inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

ads = ads_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=ads_dataset["train"].column_names
)
ads = ads.flatten()

Map (num_proc=4):   0%|          | 0/8766 [00:00<?, ? examples/s]

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4005: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4005: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4005: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 

Map (num_proc=4):   0%|          | 0/45 [00:00<?, ? examples/s]

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4005: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4005: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4005: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [9]:
from transformers import DataCollatorForSeq2Seq

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [10]:
class MetricComputer:
  def __init__(self, batch_size=8):
    self.batch_size = batch_size
    self.generations = []

  def __call__(self, eval_preds):
    ev = Evaluator(val_set, model=model, tokenizer=tokenizer, batch_size=self.batch_size, seq_tokens=True, json_processor=json_proc)
    stats = ev.calc_bleu_batched()
    self.generations.append(ev.generate_samples_batched(count=20))
    # clear_output()
    return stats

In [11]:
n_epochs = 13

training_args = Seq2SeqTrainingArguments(
  output_dir="fredT5-large-checkpoints",
  # overwrite_output_dir=True,
  evaluation_strategy="steps",
  eval_steps=2500,
  learning_rate=4e-5,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=2,
  weight_decay=0.01,
  save_total_limit=1,
  num_train_epochs=n_epochs,
  # predict_with_generate=True,
  # generation_max_length=128,
  lr_scheduler_type="cosine",
  group_by_length=False,
  warmup_steps=3,
  # fp16=True,
)


mc = MetricComputer(batch_size=8)
empty_dataset = Dataset.from_dict({"Text": [], "json": []})
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=ads["train"],
  eval_dataset=ads["test"],
  # eval_dataset=empty_dataset,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=mc,
)

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/training_args.py:1493: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer.train()

Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 31.75 GiB total capacity; 20.74 GiB already allocated; 6.94 MiB free; 20.83 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
output_dir = "fredT5-large-trained-text-token"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('fredT5-large-trained/tokenizer_config.json',
 'fredT5-large-trained/special_tokens_map.json',
 'fredT5-large-trained/vocab.json',
 'fredT5-large-trained/merges.txt',
 'fredT5-large-trained/added_tokens.json',
 'fredT5-large-trained/tokenizer.json')

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained(output_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(output_dir).to('cuda')

In [ ]:
# distill_data = pd.read_csv('../data/distill_data.csv', index_col=0)
# distill_data.head()

In [ ]:
# ev = Evaluator(distill_data)
# output = ev.generate_samples_batched(count=20000, batch_size=256)